In [29]:

def solveSystem(f1,f2,x0,y0,z0,xf,n):

    h = (xf-x0)/(n-1)
    x = np.linspace(x0,xf,n)
    y = np.zeros([n,1])
    z = np.zeros([n,1])

    y[0] = y0
    z[0] = z0

    for i in range(1,n):
        k1 = h*f1(x[i-1],y[i-1],z[i-1])
        l1 = h*f2(x[i-1],y[i-1],z[i-1])
        k2 = h*f1(x[i-1]+h/2,y[i-1]+k1/2,z[i-1]+l1/2)
        l2 = h*f2(x[i-1]+h/2,y[i-1]+k1/2,z[i-1]+l1/2)
        k3 = h*f1(x[i-1]+h/2,y[i-1]+k2/2,z[i-1]+l2/2)
        l3 = h*f2(x[i-1]+h/2,y[i-1]+k2/2,z[i-1]+l2/2)
        k4 = h*f1(x[i-1]+h, y[i-1]+k3,z[i-1]+l3)
        l4 = h*f2(x[i-1]+h, y[i-1]+k3,z[i-1]+l3)

        y[i] = y[i-1] + (k1 + 2*k2 + 2*k3 + k4)/6
        z[i] = z[i-1] + (l1 + 2*l2 + 2*l3 + l4)/6

    return np.row_stack((np.transpose(y),np.transpose(z)))

In [30]:
# Shooting Method
import pandas as pd
"""
# INPUT : y'' = p(x)y' + q(x)y + r(x) ,  y(a) = alpha, y(b) = beta
         p(x)
         q(x)
         r(x)
         a,b
         alpha,beta
# OUTPUT : approximations w1,i to y(xi)
                          w2,i to y'(xi)  to all i = 0, ... ,N
"""

import numpy as np

import matplotlib.pyplot as plt

def shootingMethod(p,q,r,a,b,alpha,beta,n):

    # n subintervals

    h = (b-a)/n
    u = np.zeros([2,n+1])
    v = np.zeros([2,n+1])
    x = np.linspace(a,b,n+1)

    #Apply Runge -  Kutta to u1,i+1  u2,i+1  v1,i+1  v2,i+1
    u = solveSystem(fun1,fun2(p,q,r),a,alpha,0,b,n+1)
    v = solveSystem(fun1,fun2(p,q,r),a,0,1,b,n+1)

    w = np.zeros([2,n+1])
    w[0,0] = alpha
    w[1,0] = (beta-u[0,n])/v[0,n]

    for i in range(1,n):
        w[0,i] = u[0,i] + w[1,0]*v[0,i]
        w[1,i] = u[1,i] + w[1,0]*v[1,i]

    return(x,w)


"""
u'' = p(x)u' + q(x)u + r(x)
u'  = n  -> n(a)  = alpha
u'' = n' -> n'(a) = 0
"""
def fun1(x, u, n):
    return n

"""
v'' = p(x)u' + q(x)u + r(x)
v'  = n  -> n(a)  = 0
v'' = n' -> n'(a) = 1
"""
def fun2(p, q, r):
    def fun22(x,v,n):
        return p(x)*n + q(x)*v + r(x)
    return fun22


# Modify p(x) q(x) and r(x) to use.

def p(x):
    return -2/x

def q(x):
    return 2/x**2

def r(x):
    return np.sin(np.log(x))/x**2

def main():

    a = 1.0
    b = 2.0
    alpha = 1
    beta = 0
    n = 10
    x,w = shootingMethod(p,q,r,a,b,alpha,beta,n) # To use in general
    #print(x)
    print(w)
    #df = pd.DataFrame({ 'w_i' : w})
    #print(pd)
    #plt.plot(x,w[0],label='Y')
    #plt.plot(x,w[1],label='Y\'')
    #plt.xlabel("X values")
    #plt.ylabel("Y values")
    #plt.legend()
    #plt.title("Solution with shooting method")
    #plt.show()

main()

[[ 1.          0.79710776  0.63928429  0.51248124  0.40759318  0.31852948
   0.24110591  0.17238143  0.11024762  0.0531667   0.        ]
 [-2.31988765 -1.77478439 -1.40473412 -1.14629818 -0.96149437 -0.8266806
  -0.72664701 -0.65132708 -0.59389248 -0.54961056  0.        ]]


In [1]:
#Input
import numpy as np
a = 1
b = 2
alpha = 1/2
beta = np.log(2)
N = 10

def p(x):
    return -4/x
def q(x):
    return -2/x**2
def r(x):
    return  (2*np.log(x))/x**2



In [3]:
#method
h = (b-a)/N
u2=np.zeros(N+2)
u1=np.zeros(N+2)
v1=np.zeros(N+2)
v2=np.zeros(N+2)
u1[0]= alpha
u2[0] = 0
v1[0] = 0
v2[0] = 1

print(h)


for i in range(0,N+1):
    x = a + i * h
    k11 = h * u2[i]
    k12 = h * ( p(x) * u2[i] + q(x) * u1[i] + r(x) )

    k21 = h * ( u2[i] + k12/2 )
    k22 = h * ( p(x + h/2) * ( u2[i] + k12/2 ) + q(x + h/2) * ( u1[i] + k11/2 ) + r(x + h/2) )

    k31 = h * ( u2[i] + k22/2 )
    k32 = h * ( p(x + h/2) * ( u2[i] + k22/2 ) + q(x + h/2) * ( u1[i] + k21/2 ) + r(x + h/2) )

    k41 = h * ( u2[i] + k32 )
    k42 = h * ( p(x + h) * ( u2[i] + k32 ) + q(x + h)*( u1[i] + k31 ) + r(x + h) )

    u1[i+1] = u1[i] + ( k11 + 2*k21 + 2*k31 + k41 )/6
    u2[i+1] = u2[i] + ( k12 + 2*k22 + 2*k32 + k42 )/6

    k_prime11 = h * v2[i]
    k_prime12 = h * ( p(x) * v2[i] + q(x) * v1[i] )

    k_prime21 = h * ( v2[i] + k_prime12/2 )
    k_prime22 = h * ( p(x + h/2) * ( v2[i] + k_prime12/2 ) + q(x + h/2) * ( v1[i] + k_prime11/2 ) )

    k_prime31 = h * ( v2[i] + k_prime22/2 )
    k_prime32 = h * ( p(x + h/2) * ( v2[i] + k_prime22/2 ) + q(x + h/2) * ( v1[i] + k_prime21/2 ) )

    k_prime41 = h * ( v2[i] + k_prime32 )
    k_prime42 = h * ( p(x + h) * ( v2[i] + k_prime32 ) + q(x + h)*( v1[i] + k_prime31) )

    v1[i+1] = v1[i] + (k_prime11 + 2*k_prime21 + 2*k_prime31 + k_prime41)/6
    v2[i+1] = v2[i] + (k_prime12 + 2*k_prime22 + 2*k_prime32 + k_prime42)/6

    #print("x_i \t u1_i \t v1_i \n{:.2f}   {:.5f}   {:.5f}".format(x,u1[i],v1[i]))
w1=np.zeros(N+2)
w2=np.zeros(N+2)
w1[0] = alpha
w2[0] = (beta - u1[-2])/(v1[-2])
#print(a,w1[0],w2[0])
print("----------------")
#import pandas as pd
for i in range(0,N+1):
    x = a + i*h
    W1 = u1[i] + w2[0]*v1[i]
    W2 = u2[i] + w2[0]*v2[i]
    print('x_i \t   W1 \t  W2 \n{:.2f}   {:.5f}   {:.5f}'.format(x ,W1,W2))



0.1
x_i 	 u1_i 	 v1_i 
1.00   0.50000   0.00000
x_i 	 u1_i 	 v1_i 
1.10   0.49613   0.08263
x_i 	 u1_i 	 v1_i 
1.20   0.48787   0.13887
x_i 	 u1_i 	 v1_i 
1.30   0.47833   0.17749
x_i 	 u1_i 	 v1_i 
1.40   0.46912   0.20406
x_i 	 u1_i 	 v1_i 
1.50   0.46101   0.22221
x_i 	 u1_i 	 v1_i 
1.60   0.45437   0.23436
x_i 	 u1_i 	 v1_i 
1.70   0.44931   0.24220
x_i 	 u1_i 	 v1_i 
1.80   0.44581   0.24690
x_i 	 u1_i 	 v1_i 
1.90   0.44379   0.24930
x_i 	 u1_i 	 v1_i 
2.00   0.44314   0.24999
----------------
x_i 	   W1 	  W2 
1.00   0.50000   1.00004
x_i 	   W1 	  W2 
1.10   0.57876   0.60865
x_i 	   W1 	  W2 
1.20   0.62674   0.37046
x_i 	   W1 	  W2 
1.30   0.65583   0.22311
x_i 	   W1 	  W2 
1.40   0.67319   0.13126
x_i 	   W1 	  W2 
1.50   0.68323   0.07413
x_i 	   W1 	  W2 
1.60   0.68874   0.03911
x_i 	   W1 	  W2 
1.70   0.69152   0.01835
x_i 	   W1 	  W2 
1.80   0.69272   0.00689
x_i 	   W1 	  W2 
1.90   0.69310   0.00148
x_i 	   W1 	  W2 
2.00   0.69315   0.00002
